In [27]:
import numpy as np
import tensorflow as tf
import gym

In [68]:
env = gym.make('FrozenLake-v0')

In [69]:
# 16 grid spaces
inputs = tf.placeholder(shape=[1,16], dtype=tf.float32)

# Each of the 16 states has 4 possible outcomes
weights = tf.Variable(tf.random_uniform([16,4], minval=0, maxval=0.1))
Q1 = tf.matmul(inputs, weights)

# Next state is chosen by maximising the dot product
output = tf.argmax(Q1, axis=1)

In [70]:
Q2 = tf.placeholder(shape=[1,4], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Q2 - Q1))
gdo = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updatedweights = gdo.minimize(loss)

In [73]:
gamma = 0.9
epsilon = 0.1
num_episodes = 50

totalReward = 0

with tf.Session() as sess:
    init = tf.initializers.global_variables()
    sess.run(init)
    
    for episode in range(num_episodes):
        state_now = env.reset()
        done = False
        reward = 0
        
        path = np.zeros(100)
        
        for j in range(100):
            feed_dict = {inputs: [np.eye(16)[state_now]]}
            path[j] = np.argmax(np.eye(16)[state_now])+ 1
            wants = [output, Q1]
            action, Y = sess.run(wants, feed_dict=feed_dict)
            if epsilon > np.random.rand(1):
                action[0] = env.action_space.sample()
                epsilon -= 10**-3
                
            state_next, reward, done, _ = env.step(action[0])
            feed_dict = {inputs: [np.eye(16)[state_next]]}
            wants = Q1
            Y1 = sess.run(wants, feed_dict=feed_dict)
            change_Y = Y
            change_Y[0, action[0]] = reward + gamma*np.max(Y1)
            
            feed_dict = {inputs: [np.eye(16)[state_now]], \
                         Q2: change_Y}
            wants = [updatedweights, weights]
            _, new_weights = sess.run(wants, feed_dict=feed_dict)
            
            totalReward += reward
            state_now = state_next
            if reward == 1:
                print('Episode {} successful'.format(episode))
                
        print(path)

[1. 2. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6.]
[1. 2. 2. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6.]
[1. 1. 2. 2. 2. 2. 1. 1. 2. 3. 4. 4. 3. 7. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.
 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.
 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.
 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.
 8. 8. 8. 8.]
[ 1.  2.  3.  7. 11. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.
 12. 12. 

[ 1.  5.  9. 10. 14. 10. 11.  7.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.]
[1. 1. 1. 1. 1. 2. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6.]
[1. 1. 1. 2. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6